In [6]:
include("../src/KagomeTransport.jl");
using .KagomeTransport.Kagome_Hamiltonian;
using LinearAlgebra;
using StaticArrays;


In [8]:
"""
Calculate the xy comp. of the complex quantum metric (both the real quantum geometry and berry curvature of the bands at a single point in the BZ)
z1, z2 ∈ U(1) = S^1, so they are complex numbers of modulus one. 
They are related to the k variables using zi = e^{2 pi i ki}
This Function is for the monolayer Kagome model 
"""
function Complex_Metric_xy(z1::ComplexF64, z2::ComplexF64, H::Matrix{ComplexF64}, p::Params, Vx, Vy,
    T1u::SMatrix{ComplexF64}, T2u::SMatrix{ComplexF64}, T3u::SMatrix{ComplexF64},
    Num_Bands::Int)

    (E, U) = eigen(Hermitian(H(z1, z2, p, T1u, T2u, T3u)))

    ωmn = E' .- E
    ωmn[ωmn .== 0] .= 1e-15
    
    Vxb = U' * Vx(z1, z2, p, T1u, T2u, T3u) * U
    Vyb = U' * Vy(z1, z2, p, T1u, T2u, T3u) * U
    
    setindex!.(Ref(Vxb), 0.0 + im * 0.0, 1:Num_Bands, 1:Num_Bands)
    setindex!.(Ref(Vyb), 0.0 + im * 0.0, 1:Num_Bands, 1:Num_Bands)

    Xmn_x = -im * (Vxb ./ ωmn); Xmn_y = -im * (Vyb ./ ωmn);
    Xmn_x * Xmn_y
end


Complex_Metric_xy